In [17]:
# Импорт библиотек
import requests as rq
from bs4 import BeautifulSoup as bs
import pandas as pd

In [70]:
class rbc_parser:
    def __init__(self):
        pass

    def get_search_table(self, param_dict: dict,
                         include_text: bool = True) -> pd.DataFrame:
        """
        Возвращает pd.DataFrame со списком статей

        includeText: bool
        ### Если True, статьи возвращаются с текстами
        """

        r = rq.get("https://www.rbc.ru/v10/search/ajax/", param_dict)
        search_table = pd.DataFrame(r.json()['items'])
        if include_text and not search_table.empty:
            get_text = lambda x: self.get_article_data(x['fronturl'])
            search_table[['overview', 'text']] = search_table.apply(get_text,
                                                                    axis=1).tolist()
        search_table["publish_date_t"] = search_table["publish_date_t"].astype(int)
        return search_table.sort_values('publish_date_t', ignore_index=True)


    def get_article_data(self, url: str):
        """
        Возвращает описание и текст статьи по ссылке
        """
        r = rq.get(url)
        soup = bs(r.text, features="lxml") # features="lxml" чтобы не было warning
        div_overview = soup.find('div', {'class': 'article__text__overview'})

        overview = None
        text = None

        if div_overview:
            overview = div_overview.text.replace('<br />','\n').strip()
        p_text = soup.find_all('p')
        if p_text:
            text = ' '.join(map(lambda x:
                                x.text.replace('<br />','\n').strip(),
                                p_text))

        return overview, text

    def get_articles(self, param_dict) -> pd.DataFrame:
        """
        Функция для скачивания статей интервалами через каждые time_step дней
        Делает сохранение таблицы через каждые save_every * time_step дней

        param_dict: dict
        ### Параметры запроса
        ###### project - раздел поиска, например, rbcnews
        ###### category - категория поиска, например, TopRbcRu_economics
        ###### dateFrom - с даты
        ###### dateTo - по дату
        ###### offset - смещение поисковой выдачи
        ###### limit - лимит статей, максимум 100
        ###### query - поисковой запрос (ключевое слово), например, РБК
        """

        offset = param_dict["offset"]
        limit = param_dict["limit"]
        step = 10
        request_params = param_dict.copy()
        request_params["limit"] = step

        out = pd.DataFrame()

        while offset < limit:
            request_params["offset"] = offset

            print(f'Parsing articles from {offset} to {offset+step}')
            out = pd.concat([out, self.get_search_table(request_params)], ignore_index=True)
            print(f"Total news: {len(out)}")
            offset += step

        print('Finish')

        return out

In [71]:
# Задаем параметры запросы и складываем в param_dict
param_dict = {'query'   : 'РБК',
            'project' : 'rbcnews',
            'category': 'TopRbcRu_economics',
            'offset'  : 0,
            'limit'   : 200,
            'material': ''}


print("param_dict:", param_dict)

РБК - param_dict: {'query': 'РБК', 'project': 'rbcnews', 'category': 'TopRbcRu_economics', 'offset': 0, 'limit': 200, 'material': ''}


In [72]:
# Действует ограничение в 10 статей на 1 запрос (параметром limit)

parser = rbc_parser()
tbl = parser.get_search_table(param_dict,
                              include_text= True) # Парсить текст статей
print(len(tbl))
tbl.head()
# Пример работы программы итеративного сбора большого количества текстов статей
# Работает, конечно, очень долго :(
table = parser.get_articles(param_dict = param_dict,
                            time_step = 1, # Шаг - 7 дней, можно больше
                            save_table=False)
table.to_csv("rbk_last_news.csv")
table.head()

10
inited
parse rdy
Parsing articles from 0 to 10
10
Parsing articles from 10 to 20
20
Parsing articles from 20 to 30
30
Parsing articles from 30 to 40
40
Parsing articles from 40 to 50
50
Parsing articles from 50 to 60
60
Parsing articles from 60 to 70
70
Parsing articles from 70 to 80
80
Parsing articles from 80 to 90
90
Parsing articles from 90 to 100
100
Parsing articles from 100 to 110
110
Parsing articles from 110 to 120
120
Parsing articles from 120 to 130
130
Parsing articles from 130 to 140
140
Parsing articles from 140 to 150
150
Parsing articles from 150 to 160
160
Parsing articles from 160 to 170
170
Parsing articles from 170 to 180
180
Parsing articles from 180 to 190
190
Parsing articles from 190 to 200
200
Finish
200


,id,fronturl,publish_date_t,publish_date,title,photo,project,category,opinion_authors,authors,anons,overview,text
0,63416c8f9a7947a5bb8af911,https://www.rbc.ru/politics/08/10/2022/63416c8f9a7947a5bb8af911,1665233340,"Sat, 08 Oct 2022 15:49:00 +0300",Минобороны заявило об отражении атак ВСУ на криворожском направлении,{'url': 'https://s0.rbk.ru/v6_top_pics/resized/250xH/media/img/7/96/756652325707967.jpg'},None,Политика,None,None,"... -Марксовское и Малая Сейдеминуха Херсонской области, но она была отбита. [ <b>РБК</b> ] В Чечне назвали неудачными попытки наступления ВСУ в Херсонской области ...","По данным Минобороны России, также отражены наступления вооруженных сил Украины на купянском, краснолиманском, южнодонецком и запорожском направлениях","Российские военные отразили попытку атаки вооруженных сил Украины на николаево-криворожском направлении, сообщает Минобороны России. По данным ведомства, три батальонные тактические группы, танков..."
1,63416d539a7947a5fd20186b,https://sportrbc.ru/news/63416d539a7947a5fd20186b,1665235864,"Sat, 08 Oct 2022 16:31:04 +0300",Навка оценила шансы Туктамышевой на попадание в тройку Гран-при России,{'url': 'https://s0.rbk.ru/v6_top_pics/resized/250xH/media/img/1/10/756652323587101.jpg'},РБК Спорт,None,None,[Никита Плохих],"... , чтобы занять первое место на Гран-при России. Об этом «<b>РБК</b> Спорт» заявила олимпийская чемпионка 2006 года в танцах на льду ...","Олимпийская чемпионка в танцах на льду отметила, что Елизавета Туктамышева показала достойный результат на турнире в Санкт-Петербурге","Получайте рассылку с новостями, которые касаются каждого. Самое актуальное о личных финансах, карьере и безопасности. Читайте РБК без рекламных баннеров на сайте и в мобильном приложении. У россий..."
2,634176869a7947a88a3ba134,https://www.rbc.ru/rbcfreenews/634176869a7947a88a3ba134,1665237198,"Sat, 08 Oct 2022 16:53:18 +0300",В Германии из-за перерезанного кабеля возникли перебои в движении поездов,{'url': 'https://s0.rbk.ru/v6_top_pics/resized/250xH/media/img/5/14/756652354935145.jpg'},None,Общество,None,None,"... , уточнив, что подрыв кабелей может вызвать помехи в радиосвязи поезда. [ <b>РБК</b> ] Три человека погибли при крушении поезда в Баварии Поезда не ...",None,"На севере Германии из-за поврежденных кабелей в двух местах почти на три часа останавливали движение поездов, оно возобновилось, но перебои на некоторых участках дороги еще сохраняются, сообщает T..."
3,63416fd19a7947a7f465cbf4,https://sportrbc.ru/news/63416fd19a7947a7f465cbf4,1665237368,"Sat, 08 Oct 2022 16:56:08 +0300",Экс-чемпион мира Лебедев назвал лучший бой чемпионата России по боксу,{'url': 'https://s0.rbk.ru/v6_top_pics/resized/250xH/media/img/7/16/756652332187167.jpg'},РБК Спорт,None,None,[Никита Плохих],... на «Фонбет чемпионате России» по боксу 2022 года. Об этом «<b>РБК</b> Спорт» заявил бывший чемпион мира по боксу по версии «Сегодня ...,"Денис Лебедев отметил, что все финальные поединки «Фонбет чемпионата России» по боксу оказались крайне интересными для зрителей. Лучшим он счел бой между Коолем и Идиговым","Получайте рассылку с новостями, которые касаются каждого. Самое актуальное о личных финансах, карьере и безопасности. Читайте РБК без рекламных баннеров на сайте и в мобильном приложении. Финальны..."
4,6341822a9a7947a7213c2e61,https://kuban.rbc.ru/krasnodar/freenews/6341822a9a7947a7213c2e61,1665237552,"Sat, 08 Oct 2022 16:59:12 +0300",Возобновлено автомобильное движение по Крымскому мосту,{'url': ''},None,None,None,None,"... железнодорожной части моста обещают запустить к 20:00, сообщал <b>РБК</b>. Как сообщал <b>РБК</b> Краснодар, движение по Крымскому мосту прекратилось с утра 8 ...",None,"По Крымскому мосту возобновили движение автомобилей, заявил глава Крыма Сергей Аксенов. Пропускают легковушки и автобусы, их досматривают. Водителей грузовиков просят пользоваться паромной перепра..."


In [73]:
pd.read_csv("rbk_last_news.csv", index_col=0)

,id,fronturl,publish_date_t,publish_date,title,photo,project,category,opinion_authors,authors,anons,overview,text
0,63416c8f9a7947a5bb8af911,https://www.rbc.ru/politics/08/10/2022/63416c8f9a7947a5bb8af911,1665233340,"Sat, 08 Oct 2022 15:49:00 +0300",Минобороны заявило об отражении атак ВСУ на криворожском направлении,{'url': 'https://s0.rbk.ru/v6_top_pics/resized/250xH/media/img/7/96/756652325707967.jpg'},NaN,Политика,NaN,NaN,"... -Марксовское и Малая Сейдеминуха Херсонской области, но она была отбита. [ <b>РБК</b> ] В Чечне назвали неудачными попытки наступления ВСУ в Херсонской области ...","По данным Минобороны России, также отражены наступления вооруженных сил Украины на купянском, краснолиманском, южнодонецком и запорожском направлениях","Российские военные отразили попытку атаки вооруженных сил Украины на николаево-криворожском направлении, сообщает Минобороны России. По данным ведомства, три батальонные тактические группы, танков..."
1,63416d539a7947a5fd20186b,https://sportrbc.ru/news/63416d539a7947a5fd20186b,1665235864,"Sat, 08 Oct 2022 16:31:04 +0300",Навка оценила шансы Туктамышевой на попадание в тройку Гран-при России,{'url': 'https://s0.rbk.ru/v6_top_pics/resized/250xH/media/img/1/10/756652323587101.jpg'},РБК Спорт,NaN,NaN,['Никита Плохих'],"... , чтобы занять первое место на Гран-при России. Об этом «<b>РБК</b> Спорт» заявила олимпийская чемпионка 2006 года в танцах на льду ...","Олимпийская чемпионка в танцах на льду отметила, что Елизавета Туктамышева показала достойный результат на турнире в Санкт-Петербурге","Получайте рассылку с новостями, которые касаются каждого. Самое актуальное о личных финансах, карьере и безопасности. Читайте РБК без рекламных баннеров на сайте и в мобильном приложении. У россий..."
2,634176869a7947a88a3ba134,https://www.rbc.ru/rbcfreenews/634176869a7947a88a3ba134,1665237198,"Sat, 08 Oct 2022 16:53:18 +0300",В Германии из-за перерезанного кабеля возникли перебои в движении поездов,{'url': 'https://s0.rbk.ru/v6_top_pics/resized/250xH/media/img/5/14/756652354935145.jpg'},NaN,Общество,NaN,NaN,"... , уточнив, что подрыв кабелей может вызвать помехи в радиосвязи поезда. [ <b>РБК</b> ] Три человека погибли при крушении поезда в Баварии Поезда не ...",NaN,"На севере Германии из-за поврежденных кабелей в двух местах почти на три часа останавливали движение поездов, оно возобновилось, но перебои на некоторых участках дороги еще сохраняются, сообщает T..."
3,63416fd19a7947a7f465cbf4,https://sportrbc.ru/news/63416fd19a7947a7f465cbf4,1665237368,"Sat, 08 Oct 2022 16:56:08 +0300",Экс-чемпион мира Лебедев назвал лучший бой чемпионата России по боксу,{'url': 'https://s0.rbk.ru/v6_top_pics/resized/250xH/media/img/7/16/756652332187167.jpg'},РБК Спорт,NaN,NaN,['Никита Плохих'],... на «Фонбет чемпионате России» по боксу 2022 года. Об этом «<b>РБК</b> Спорт» заявил бывший чемпион мира по боксу по версии «Сегодня ...,"Денис Лебедев отметил, что все финальные поединки «Фонбет чемпионата России» по боксу оказались крайне интересными для зрителей. Лучшим он счел бой между Коолем и Идиговым","Получайте рассылку с новостями, которые касаются каждого. Самое актуальное о личных финансах, карьере и безопасности. Читайте РБК без рекламных баннеров на сайте и в мобильном приложении. Финальны..."
4,6341822a9a7947a7213c2e61,https://kuban.rbc.ru/krasnodar/freenews/6341822a9a7947a7213c2e61,1665237552,"Sat, 08 Oct 2022 16:59:12 +0300",Возобновлено автомобильное движение по Крымскому мосту,{'url': ''},NaN,NaN,NaN,NaN,"... железнодорожной части моста обещают запустить к 20:00, сообщал <b>РБК</b>. Как сообщал <b>РБК</b> Краснодар, движение по Крымскому мосту прекратилось с утра 8 ...",NaN,"По Крымскому мосту возобновили движение автомобилей, заявил глава Крыма Сергей Аксенов. Пропускают легковушки и автобусы, их досматривают. Водителей грузовиков просят пользоваться паромной перепра..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,634113979a7947913b7b32c8,https://www.rbc.ru/society/08/10/2022/634113979a7947913